#### 中文数据集

In [3]:
from datasets import load_dataset

##Azure99___blossom-math-v1数据集
dataset = load_dataset(r"/root/.cache/huggingface/datasets/Azure99___blossom-math-v1")
print(dataset)


/root/miniconda3/envs/glm4-9bchat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'output', 'answer'],
        num_rows: 10000
    })
})


In [4]:
print(dataset['train'][0:100])

{'id': ['1', '5', '6', '7', '8', '9', '10', '12', '18', '19', '20', '21', '22', '23', '24', '27', '28', '30', '31', '36', '37', '38', '39', '40', '46', '48', '50', '52', '54', '55', '59', '62', '63', '64', '65', '67', '68', '69', '71', '72', '77', '78', '81', '82', '85', '87', '90', '94', '95', '97', '98', '101', '102', '103', '106', '109', '110', '111', '114', '115', '118', '120', '122', '124', '125', '126', '127', '129', '134', '135', '139', '143', '147', '148', '149', '150', '151', '152', '158', '160', '172', '176', '177', '181', '185', '192', '193', '194', '196', '197', '202', '206', '207', '209', '211', '212', '213', '216', '219', '222'], 'input': ['镇海雅乐学校二年级的小朋友到一条小路的一边植树．小朋友们每隔2米种一棵树（马路两头都种了树），最后发现一共种了11棵，这条小路长多少米．', '小刚的体重是28.4千克，小强的体重是小刚的1.4倍，小强的体重=多少千克？', '10.6-0.4与5.5的积，所得的差除以2.1，商=？', '象山学校原有学生700人，7月份毕业280人，9月份招入新生350人，这时象山学校的学生比原来增加百分之几？', '买一套住房，售价28万元，每年付2万元，多少年后才能付清？', '王叔叔开车从甲地去乙地，以70千米/时的速度行了6小时才到达．返回时由于有任务，加快了速度，只用了4小时就回到了甲地，返回时的速度=？', '一个果园的李树棵数是桃树的(7/8)，桃树棵数是梨树的(5

In [2]:
import os
import json
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer

# ========== 1. 加载数据集 ==========

# 假设你已经本地/远程存储了一个包含以下字段的数据集:
#   ['id', 'input', 'output', 'answer']
# 并且 num_rows = 10000
# 这里做演示，可能需要替换成你实际的数据集加载方式
# 例如： ds = load_dataset("json", data_files="your_data.json")
#       ds = ds["train"]  # 如果只有一个 split
#       或者 ds = load_from_disk("your_dataset_dir")
# 下面仅作示例，假设 ds 已经是 datasets.Dataset 对象
#
# ds = load_dataset("json", data_files={"train": "your_data.json"})
# ds = ds["train"]
# 
# 如果你已有 DatasetDict 结构：
# ds_all = load_dataset("json", data_files={"train": "your_data.json", "test": "your_test_data.json"})
# ds = ds_all["train"]

# 这里我们用一个伪造的示例:
fake_data = [
    {
        "id": 1,
        "input": "勾股定理的详细推导是什么？",
        "output": "勾股定理主要阐述了直角三角形的三边关系 a^2 + b^2 = c^2 ...",
        "answer": "典型举例是3,4,5三元组，推导方法包括相似三角形、面积拼接等..."
    },
    {
        "id": 2,
        "input": "如何快速排序一个数列？",
        "output": "快速排序是一种分而治之的算法，平均时间复杂度 O(n log n)...",
        "answer": "将数组以枢纽值分割，然后递归处理子数组，效率较高。"
    }
]
# 将其转换成 datasets.Dataset
from datasets import Dataset
ds = Dataset.from_list(fake_data)

# ========== 2. 加载大语言模型 ==========

model_path = "/root/autodl-tmp/model/Qwen2-math7B"  # 替换为你的模型目录或Huggingface名称
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"  # 当使用多GPU/自动分配时，可用"auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 如果你的 tokenizer 没有 `apply_chat_template` 方法，需要手动写:
# 下面只是说明 Qwen 系列可能具备apply_chat_template，自行确认
if not hasattr(tokenizer, "apply_chat_template"):
    # 定义一个简单的替代函数
    def apply_chat_template(messages, tokenize=False, add_generation_prompt=True):
        # 这里你需要根据模型对多轮对话格式的要求来拼接
        # 简单示例：每条消息用 <role>: content\n
        text_parts = []
        for msg in messages:
            role = msg["role"]
            content = msg["content"]
            text_parts.append(f"<{role}>: {content}")
        if add_generation_prompt:
            text_parts.append("<assistant>:")
        full_text = "\n".join(text_parts)
        return full_text
    tokenizer.apply_chat_template = apply_chat_template

# ========== 3. 构造 Prompt 让模型生成 GoT(JSON) ==========

def build_prompt(input_text, output_text, final_answer):
    """
    构造一个系统/用户提示，期望模型输出具备以下 JSON 结构：
    {
      "conversation_id": ...,
      "messages": [
         {
           "role": "user",
           "content": ...
         },
         {
           "role": "assistant",
           "content": ...,
           "graph_of_thought": {...},
           "final_answer": ...
         }
      ],
      "summaries": [],
      "global_summary": ...
    }
    """
    system_content = (
        "You are a helpful assistant. "
        "Please produce a conversation in JSON format that includes:\n"
        "- conversation_id (string)\n"
        "- an array 'messages' with role=user or role=assistant\n"
        "- an object 'graph_of_thought' under the assistant message\n"
        "- a 'final_answer' under the assistant message\n"
        "- optional 'summaries' or 'global_summary'\n"
        "Ensure the JSON is valid and well-structured."
    )
    
    # 你可以将用户的 input/output/answer 都融入 User Prompt，让模型去组合。
    # 也可以把 output/answer 放到system指令里，看你需要哪种效果。
    user_content = (
        f"User question: {input_text}\n\n"
        f"Model standard solution (reference): {output_text}\n\n"
        f"Final short answer (reference): {final_answer}\n\n"
        "Now produce a conversation JSON with a graph_of_thought (nodes, edges, reflexion) "
        "and a final_answer field that concisely solves the user question. "
        "The conversation_id can be any unique string. "
        "You can refine or expand the standard solution as needed."
    )
    
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]
    
    # 使用 Qwen 提供的模板方法
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True  # 在结尾自动添加 <assistant> 标记
    )
    return text

# ========== 4. 遍历数据集并调用 LLM 生成 JSON ==========

new_data = []
max_new_tokens = 25000

for row in ds:
    input_text = row["input"]
    output_text = row["output"]
    answer_text = row["answer"]
    
    # 构造 Prompt
    prompt_text = build_prompt(input_text, output_text, answer_text)
    
    # 构造模型输入
    model_inputs = tokenizer([prompt_text], return_tensors="pt")
    model_inputs = {k: v.to(device) for k, v in model_inputs.items()}
    
    # 推理
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False  # 这里用greedy，可根据需要改为True并设置temperature等参数
    )
    
    # 去掉前面prompt的部分，只拿assistant新增的tokens
    input_ids = model_inputs["input_ids"][0]
    output_ids = generated_ids[0][len(input_ids):]
    
    response_text = tokenizer.decode(output_ids, skip_special_tokens=True)
    
    # 试图将模型输出解析为JSON，如果失败就存原始文本
    try:
        response_json = json.loads(response_text)
    except Exception:
        # 如果模型未严格输出JSON，这里可以做进一步清洗或仅保留原始文本
        response_json = {"raw_text": response_text}
    
    new_data.append({
        "id": row["id"],
        "conversation_json": response_json
    })

# ========== 5. 将结果保存到新数据集或文件 ==========

# 你可以直接把 new_data 转成一个 Dataset
new_dataset = Dataset.from_list(new_data)

# 如果想要保存为本地 JSON，可以执行:
output_file = r"/root/autodl-tmp/code/2025service_creativity/process_dataset/huggingfaceset/conversation_with_graph_of_thought.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(new_data, f, ensure_ascii=False, indent=2)

print(f"Done! 共生成 {len(new_data)} 条记录，已保存到 {output_file}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Done! 共生成 2 条记录，已保存到 /root/autodl-tmp/code/2025service_creativity/process_dataset/huggingfaceset/conversation_with_graph_of_thought.json


#### 生成GOT数据集

In [1]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# ========== 0. 基本配置 ==========

MODEL_PATH = r"/root/autodl-tmp/model/Qwen2-math7B"  # 替换为你的模型目录或模型名称
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 输出文件，用于存储多轮对话JSON
OUTPUT_FILE = r"/root/autodl-tmp/code/2025service_creativity/process_dataset/huggingfaceset/ConvGoTAzure99_blossom-math-v1.json"

# ========== 1. 加载你的数据集 ==========

# 假设 dataset 包含 train / test / validation 等 split
# 在此处只演示对 "train" split 进行处理：
dataset_all = load_dataset(r"/root/.cache/huggingface/datasets/Azure99___blossom-math-v1")
print(dataset_all)  # 查看数据集信息

# 这里假设我们只处理 train split：
ds = dataset_all["train"]

# 如果你的数据集中字段并非 "input", "output", "solution"，请自行更改下面的逻辑
# 假设 ds 中每条记录如下：
# {
#   "id": ...,         # 题目编号
#   "input": ...,      # 用户题目
#   "output": ...,     # 参考输出
#   "solution": ...    # 参考解答
# }

# ========== 2. 准备示例对话 JSON，用作“结构参考” ==========

example_got_json = r"""
{
  "conversation_id": "1",
  "messages": [
    {
      "role": "user",
      "content": "勾股定理的详细推导是什么？能不能举例说明？"
    },
    {
      "role": "assistant",
      "content": "勾股定理主要阐述了直角三角形斜边与两直角边的平方和关系...",
      "graph_of_thought": {
        "nodes": [
          { "id": "n1", "content": "回忆定义：a^2 + b^2 = c^2, c是斜边" },
          { "id": "n2", "content": "推导一：利用面积拆分方法" },
          { "id": "n3", "content": "推导二：借助相似三角形" },
          { "id": "n4", "content": "举例：3,4,5" }
        ],
        "edges": [
          { "from": "n1", "to": "n2", "relation": "alternative_proof_1" },
          { "from": "n1", "to": "n3", "relation": "alternative_proof_2" },
          { "from": "n1", "to": "n4", "relation": "example" }
        ],
        "reflexion": [
          { 
            "node_id": "n3", 
            "self_check": "相似三角形推导是否清晰？", 
            "revised_content": "可以再补充角度关系..."
          }
        ]
      },
      "final_answer": "勾股定理的基本推导方法包括面积拼接法和相似三角形法，典型例子有3,4,5三元组..."
    }
  ],
  "summaries": [],
  "global_summary": "本段对话针对勾股定理的推导过程与整数例子进行较详细讨论。"
}
""".strip()


# ========== 3. 初始化模型和分词器 ==========

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    device_map="auto"  # 如果有多GPU，可自动分配
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)


# ========== 4. 构造中文 Prompt：将 input/output/solution 合并进来 ==========

def build_prompt(record_id: str, user_input: str, std_output: str, std_solution: str) -> str:
    """
    使用中文提示，让模型输出与 example_got_json 类似的多轮对话格式。
    
    参数:
    - record_id: 题目编号或唯一ID
    - user_input: 用户提问/题目
    - std_output: 参考输出
    - std_solution: 参考解答
    """
    system_content = (
        "你是一位乐于助人的AI助手。下面有一个对话JSON的示例，它包含了以下内容：\n\n"
        f"{example_got_json}\n\n"
        "请仔细分析上面的JSON结构。现在有一个新题目，需要你产出一个类似的多轮对话JSON，要求：\n"
        "1. 包含一个独特的 \"conversation_id\"（例如可以使用题目的ID）。\n"
        "2. 在 \"messages\" 数组中，至少包含一条用户提问 (role=user) 和一条AI回答 (role=assistant)。\n"
        "3. 在 assistant 的回答中，一定要包含 \"graph_of_thought\" 字段（其中包含若干 \"nodes\", \"edges\", \"reflexion\" 等）\n"
        "4. assistant 的回答还需包含一个 \"final_answer\" 字段，用于给出最终简明结论。\n"
        "5. 你可以根据参考输出和解答内容进行总结、提炼、补充，但生成的 JSON 要**结构完整**、**逻辑严密**、**推理科学**，不要输出多余说明。\n"
        "6. 不要在 JSON 之外输出额外内容。只需返回纯 JSON。\n"
    )

    # 将题目信息、参考输出和参考解答，放在用户提示中
    user_content = (
        f"新的题目ID: {record_id}\n"
        f"题目内容 (user_input): {user_input}\n"
        f"参考输出 (std_output): {std_output}\n"
        f"参考解答 (std_solution): {std_solution}\n\n"
        "请你基于以上信息，输出一个完整的多轮对话JSON。"
    )

    # 拼接对话Prompt
    prompt_text = (
        f"<system>: {system_content}\n"
        f"<user>: {user_content}\n"
        "<assistant>:"  # 给模型一个标记，以便开始回答
    )
    return prompt_text


# ========== 5. 调用模型并尝试解析成 JSON ==========

def generate_got_json(prompt_text: str) -> dict:
    """
    给定中文 Prompt，调用模型生成文本，并尝试解析为 JSON。
    如果解析失败，则返回 { "raw_text": ... }。
    """
    inputs = tokenizer(prompt_text, return_tensors="pt").to(DEVICE)

    # 推理生成
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=False,  # 这里使用greedy，可改为True并设置温度等
            top_k=1,
        )

    # 将输出转成字符串，并截掉Prompt的部分
    input_len = inputs["input_ids"].shape[1]
    gen_ids = output_ids[0][input_len:]
    raw_output = tokenizer.decode(gen_ids, skip_special_tokens=True)

    # 尝试解析JSON
    try:
        parsed_json = json.loads(raw_output)
        return parsed_json
    except Exception:
        return {"raw_text": raw_output}


# ========== 6. 工具函数：追加数据到 JSON 文件 ==========

def append_data_to_json_file(data: dict, filename: str):
    """
    读取原文件(若有)，将新数据追加到数组中，并写回。
    文件格式: [
      { conversation_json1... },
      { conversation_json2... },
      ...
    ]
    """
    if not os.path.exists(filename):
        # 如果文件不存在，创建一个新数组并写入
        with open(filename, "w", encoding="utf-8") as f:
            json.dump([data], f, ensure_ascii=False, indent=2)
    else:
        # 文件存在，则读取后追加
        with open(filename, "r", encoding="utf-8") as f:
            try:
                old_data = json.load(f)
                if not isinstance(old_data, list):
                    old_data = []
            except:
                old_data = []

        old_data.append(data)

        with open(filename, "w", encoding="utf-8") as f:
            json.dump(old_data, f, ensure_ascii=False, indent=2)


# ========== 7. 遍历数据集并生成 GoT 对话 JSON，保存到文件 ==========

def main():
    # 这里可以先清空或初始化输出文件
    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)  # 如果想保留历史记录，可以不删除

    # 遍历数据集中的每条样本
    for i, row in enumerate(ds):
        # 假设字段为["id", "input", "output", "solution"]
        record_id = str(row["id"])
        user_input = row["input"]
        std_output = row["output"]
        std_solution = row["answer"]

        # 构造中文 prompt
        prompt_text = build_prompt(record_id, user_input, std_output, std_solution)

        # 调用模型生成
        conversation_json = generate_got_json(prompt_text)

        # 追加到输出文件
        append_data_to_json_file(conversation_json, OUTPUT_FILE)

        # 仅演示前10条，可自行去掉限制
        if i < 10:
            print(f"[INFO] 已生成第 {i+1} 条记录，conversation_id={record_id}")
        if i == 9:
            print("（仅打印前10条处理情况，其余继续处理中...）")

    print(f"处理完成！所有 GoT 对话结果保存在 {OUTPUT_FILE}。")


if __name__ == "__main__":
    main()


DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'output', 'answer'],
        num_rows: 10000
    })
})


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/root/miniconda3/envs/glm4-9bchat/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/glm4-9bchat/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


[INFO] 已生成第 1 条记录，conversation_id=1
[INFO] 已生成第 2 条记录，conversation_id=5
[INFO] 已生成第 3 条记录，conversation_id=6
[INFO] 已生成第 4 条记录，conversation_id=7
[INFO] 已生成第 5 条记录，conversation_id=8
[INFO] 已生成第 6 条记录，conversation_id=9
[INFO] 已生成第 7 条记录，conversation_id=10
[INFO] 已生成第 8 条记录，conversation_id=12
[INFO] 已生成第 9 条记录，conversation_id=18
[INFO] 已生成第 10 条记录，conversation_id=19
（仅打印前10条处理情况，其余继续处理中...）


KeyboardInterrupt: 

#### GoT花费时间过长解决办法

In [ ]:
import os
import json
import torch
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# ========== 0. 基本配置 ==========
MODEL_PATH = r"/root/autodl-tmp/model/Qwen2-math7B"  # 替换为你的模型目录或模型名称
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

OUTPUT_FILE = r"/root/autodl-tmp/code/2025service_creativity/process_dataset/huggingfaceset/ConvGoTAzure99_blossom-math-v1.json"

# 线程池最大并发数：如果仍出现问题，可改为1测试
MAX_WORKERS = 2

# ========== 1. 加载数据集 ==========
dataset_all = load_dataset(r"/root/.cache/huggingface/datasets/Azure99___blossom-math-v1")
ds = dataset_all["train"]
print(dataset_all)

# 假设 ds 每条记录包含 ["id", "input", "output", "answer"] 等字段
# 其中 "id" 可以当作 conversation_id，"input" 就是用户题目，"answer" 是参考解答

# ========== 2. 示例对话 JSON ==========
example_got_json = r"""
{
  "conversation_id": "1",
  "messages": [
    {
      "role": "user",
      "content": "示例题目：勾股定理的详细推导是什么？"
    },
    {
      "role": "assistant",
      "content": "勾股定理阐述了直角三角形斜边与两直角边的平方和关系...",
      "graph_of_thought": {
        "nodes": [
          { "id": "n1", "content": "定义 a^2 + b^2 = c^2" },
          { "id": "n2", "content": "利用面积拆分" }
        ],
        "edges": [
          { "from": "n1", "to": "n2", "relation": "alternative_proof" }
        ],
        "reflexion": [
          { 
            "node_id": "n2", 
            "self_check": "推导是否自洽？", 
            "revised_content": "可以补充更多细节..."
          }
        ]
      },
      "final_answer": "主要推导方式包括面积拼接与相似三角形..."
    }
  ],
  "global_summary": "此对话讨论了勾股定理的不同推导方法。"
}
""".strip()

# ========== 3. 初始化模型（无8-bit量化）与分词器 ==========
# 使用 FP16 或 "auto" 即可，避免 8-bit 量化导致的不兼容
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",       # 如果有多GPU，可自动分配
    torch_dtype=torch.float16 # 使用半精度，可节省显存
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# ========== 4. 构造中文 Prompt ==========
def build_prompt(record_id: str, user_input: str, std_output: str, std_solution: str) -> str:
    """
    生成与 example_got_json 类似结构的多轮对话（中文）。
    包含 conversation_id, messages, assistant 中的 graph_of_thought, final_answer 等。
    """
    system_content = (
        "你是一位乐于助人的AI助手。以下是一个对话JSON示例:\n"
        f"{example_got_json}\n\n"
        "请参照此示例结构，生成一段新的对话JSON。\n"
        "要求：\n"
        "1. 使用题目ID作为 conversation_id；\n"
        "2. 在 messages 中给出 user 的问题和 assistant 的回答；\n"
        "3. assistant 的回答必须包含 graph_of_thought 和 final_answer；\n"
        "4. 仅输出 JSON，不要额外解释。\n"
    )

    user_content = (
        f"题目ID: {record_id}\n"
        f"用户题目: {user_input}\n"
        f"参考输出: {std_output}\n"
        f"参考解答: {std_solution}\n"
        "请生成结构完备、逻辑合理的多轮对话JSON。"
    )

    prompt_text = (
        f"<system>: {system_content}\n"
        f"<user>: {user_content}\n"
        "<assistant>:"
    )
    return prompt_text

# ========== 5. 调用模型并解析 JSON ==========
def generate_got_json(prompt_text: str) -> dict:
    """
    使用 greedy 搜索 (do_sample=False)，并将输出尝试解析为 JSON。
    如果解析失败，则存入 {"raw_text": "..."}。
    """
    inputs = tokenizer(prompt_text, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,   # 适度控制生成长度，避免过长
            do_sample=False       # Greedy搜索
        )
    input_len = inputs["input_ids"].shape[1]
    gen_ids = output_ids[0][input_len:]
    raw_output = tokenizer.decode(gen_ids, skip_special_tokens=True)
    try:
        parsed_json = json.loads(raw_output)
        return parsed_json
    except:
        return {"raw_text": raw_output}

# ========== 6. 追加数据到 JSON 文件 ==========
def append_data_to_json_file(data: dict, filename: str):
    """
    读取或创建文件，向里面的数组追加一条记录。
    最终结构: [ {...}, {...}, ... ]
    """
    if not os.path.exists(filename):
        with open(filename, "w", encoding="utf-8") as f:
            json.dump([data], f, ensure_ascii=False, indent=2)
    else:
        with open(filename, "r", encoding="utf-8") as f:
            try:
                old_data = json.load(f)
                if not isinstance(old_data, list):
                    old_data = []
            except:
                old_data = []
        old_data.append(data)
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(old_data, f, ensure_ascii=False, indent=2)

# ========== 7. 并行处理数据集 ==========
def process_record(row):
    """
    单条数据处理逻辑：构造Prompt、调用模型、解析JSON、写入文件
    返回 record_id 供打印
    """
    record_id = str(row["id"])
    user_input = row["input"]
    std_output = row["output"]
    std_solution = row["answer"]  # 如果你的数据集中字段不是 answer，请自行修改

    prompt_text = build_prompt(record_id, user_input, std_output, std_solution)
    conversation_json = generate_got_json(prompt_text)
    append_data_to_json_file(conversation_json, OUTPUT_FILE)
    return record_id

def main():
    # 若存在旧文件可删除，以免重复追加
    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)

    # ThreadPoolExecutor 并行调用
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures_map = {}
        for i, row in enumerate(ds):
            future = executor.submit(process_record, row)
            futures_map[future] = i

        for future in as_completed(futures_map):
            idx = futures_map[future]
            try:
                rec_id = future.result()
                if idx < 10:
                    print(f"[INFO] 第 {idx+1} 条处理完成，conversation_id={rec_id}")
                if idx == 9:
                    print("(仅显示前10条处理日志...)")
            except Exception as e:
                print(f"[ERROR] 第 {idx+1} 条出现异常: {e}")

    print(f"全部处理完成！结果保存在 {OUTPUT_FILE}")

if __name__ == "__main__":
    main()


DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'output', 'answer'],
        num_rows: 10000
    })
})


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/root/miniconda3/envs/glm4-9bchat/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


[INFO] 第 2 条处理完成，conversation_id=5
[INFO] 第 3 条处理完成，conversation_id=6
[INFO] 第 1 条处理完成，conversation_id=1
[INFO] 第 4 条处理完成，conversation_id=7
[INFO] 第 5 条处理完成，conversation_id=8
[INFO] 第 6 条处理完成，conversation_id=9
[INFO] 第 8 条处理完成，conversation_id=12
[INFO] 第 7 条处理完成，conversation_id=10
[INFO] 第 9 条处理完成，conversation_id=18
[INFO] 第 10 条处理完成，conversation_id=19
(仅显示前10条处理日志...)
